In [14]:
import pandas as pd
import os
from glob import glob

# 文件夹路径和排除文件路径
folder_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/国标码'
exclude_file = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/国标码/china10_cnty_fromexcel_cnty_pref.csv'

# 获取所有 .xls 文件的路径，排除指定文件
all_files = glob(os.path.join(folder_path, '**/*.xls'), recursive=True)
files_to_process = [file for file in all_files if file != exclude_file]

# 定义感兴趣的字段
columns_of_interest = ['PROVGB', 'CITYGB', 'CNTYGB', 'CPROV', 'EPROV', 'CCNTY', 'ECNTY']

# 用于存储所有可能取值组合的集合
all_combinations = set()

# 遍历每个文件并提取感兴趣字段的值组合
for file in files_to_process:
    try:
        # 读取Excel文件
        df = pd.read_excel(file, usecols=columns_of_interest)
        
        # 去除缺失值的行，然后将感兴趣的字段转为元组形式
        df = df.dropna(subset=columns_of_interest)
        
        # 添加每一行的值组合到集合中去重
        for row in df[columns_of_interest].itertuples(index=False, name=None):
            all_combinations.add(row)
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# 将结果转换为DataFrame，并按第一列降序排列
all_combinations_df = pd.DataFrame(list(all_combinations), columns=columns_of_interest)
all_combinations_df = all_combinations_df.sort_values(by=columns_of_interest[0], ascending=False)

# 保存到CSV文件，不带索引
output_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241105area_value_combinations.csv'
all_combinations_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"All unique value combinations saved to {output_path}")

All unique value combinations saved to /Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241105area_value_combinations.csv


In [15]:
clean的时候加一步操作：新增一列“nozu_area”，删除掉省份以后，“族“字及其之前的字符，列“area”如果出现“族”这个字，
最终存要存clean之前的area内容
import pandas as pd
from fuzzywuzzy import process
from tqdm import tqdm  # 用于显示进度条
import re  # 用于正则表达式
from joblib import Parallel, delayed  # 用于并行处理

# 定义一个函数去除特殊字符和空格
def clean_area_name(area_name):
    area_name = re.sub(r'[^\w\s]', '', area_name)  # 去除特殊字符
    area_name = re.sub(r'\s+', '', area_name)  # 去除所有空格
    return area_name.strip()  # 去除首尾空格

# 匹配函数，增加对 name 的记录
def match_area(area, name):
    match = process.extractOne(area, reference_df['combined'])
    return name, area, match[0], match[1]  # 返回 name，原始地名，匹配结果和相似度

# 读取地名文件
area_df = pd.read_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales/地图匹配_0426_v4.csv')  # 包含不规范的地名
reference_df = pd.read_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241105area_value_combinations.csv')  # 包含所有可能的地名

columns_to_check = [
    "PROVGB_1953", "CITYGB_1953", "CNTYGB_1953",
    "PROVGB_1964", "CITYGB_1964", "CNTYGB_1964",
    "PROVGB_1982", "CITYGB_1982", "CNTYGB_1982",
    "PROVGB_1990a", "CITYGB_1990a", "CNTYGB_1990a",
    "PROVGB_2000", "CITYGB_2000", "CNTYGB_2000",
    "PROVGB_2010", "CITYGB_2010", "CNTYGB_2010"
]

# 保留所有这些列都为空的行
area_df = area_df[area_df[columns_to_check].isnull().all(axis=1)]


# 清洗地名
tqdm.pandas(desc="清洗地名")
area_df['cleaned_area'] = Parallel(n_jobs=-1)(
    delayed(clean_area_name)(area) for area in tqdm(area_df['area'], desc="清洗地名")
)

reference_df['combined'] = Parallel(n_jobs=-1)(
    delayed(clean_area_name)(f"{row['CPROV']}{row['CCNTY']}") for _, row in reference_df.iterrows()
)

# 匹配地名并记录结果
tqdm.pandas(desc="匹配地名")
results = Parallel(n_jobs=-1)(
    delayed(match_area)(row['cleaned_area'], row['name']) for _, row in tqdm(area_df.iterrows(), total=area_df.shape[0], desc="匹配地名")
)

# 将结果转换为 DataFrame
results_df = pd.DataFrame(results, columns=['name', 'original_area', 'matched_area', 'similarity'])

# 保存结果到 CSV 文件
results_df.to_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/matched_areas.csv', index=False)













匹配地名:   0%|          | 0/17263 [06:41<?, ?it/s]




匹配地名: 100%|██████████| 3348/3348 [00:46<00:00, 71.24it/s]


读取这个表：/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241105matched_areas.csv
如果, 'matched_area'的后2/3/4个字符出现在'original_area'中，similarity”一列低于60分，改为61分

In [30]:
import pandas as pd

# 读取 CSV 文件
file_path = '/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/matched_areas.csv'
df = pd.read_csv(file_path)

# 定义一个函数检查条件并修改 similarity
def update_similarity(row):
    matched_area = row['matched_area']
    original_area = row['cleaned_original_area']
    similarity = row['similarity']
    
    # 检查后2、3、4个字符是否出现在 original_area 中
    for i in range(2, 5):  # 从 2 到 4
        if matched_area[-i:] in original_area and similarity < 60:
            return 61  # 修改为 61 分
    return similarity  # 不修改，返回原值

# 应用函数并更新 similarity 列
df['similarity'] = df.apply(update_similarity, axis=1)

# 按照 similarity 列升序排序
df = df.sort_values(by='similarity', ascending=True)

# 保存修改后的 DataFrame 回 CSV 文件（可选）
df.to_csv(file_path, index=False)

# 显示更新后的 DataFrame（可选）
print(df.head())


                  name                              cleaned_original_area  \
1734   02561__554巧计定是非  吉林人做有自己的木匠铁匠也有自己的看病先生自己寻找草药自己配药有个畲族村寨有一户世代名医真是...   
2565  00349__352黄花鱼和鳖鱼  上海早很早以前黄花鱼就是海里的游泳能手黄花鱼啥都好就是有点骄傲它自称是海里游泳第一其实呢比它...   
1095     00280__283黄浦江  上海久很久以前上海还是一片荒凉的沼泽地沼泽地当中弯弯曲曲有一条河流河床很浅雨水多了泛滥成灾雨...   
2341     02404__391飞去。  吉林这样素贞化成了一只体态娇小羽毛灰白黑相间的鸟儿像披着一身朴素的孝衣这鸟儿越过高山越过河流...   
2141    06824__113 病了。  广东世藩听到消息后便到谭家探访谭大初对他说希望他协助捞回玉镯并提出自己出钱用水车车干塘水的办...   

     matched_area  similarity  
1734        吉林洮南市          24  
2565        广东花都区          24  
1095        青海久治县          24  
2341        云南宜良县          24  
2141         广东梅县          30  


删除省份名称后紧接的“上”或“下”字，以及“族”字及其之间的内容和“族”字本身：

In [18]:
provinces = [
    "北京", "天津", "河北", "山西", "内蒙古",
    "辽宁", "吉林", "黑龙江", "上海", "江苏",
    "浙江", "安徽", "福建", "江西", "山东",
    "河南", "湖北", "湖南", "广东", "广西",
    "海南", "重庆", "四川", "贵州", "云南",
    "西藏", "陕西", "甘肃", "青海", "宁夏",
    "新疆", "香港", "澳门", "台湾"
]

In [28]:
import pandas as pd
from fuzzywuzzy import process
from tqdm import tqdm  # 用于显示进度条
import re  # 用于正则表达式
from joblib import Parallel, delayed  # 用于并行处理

# 定义一个函数去除特殊字符和空格，并处理“族”字
def clean_area_name(area_name):
    # 去除特殊字符
    area_name = re.sub(r'[^\w\s]', '', area_name)  
    # 去除所有空格
    area_name = re.sub(r'\s+', '', area_name)  
    return area_name.strip()  # 去除首尾空格


def remove_province_and_zu(area_name, provinces):
    # 遍历省份名称
    for province in provinces:
        if area_name.startswith(province):
            # 使用正则表达式删除省份名称后紧接的“上”或“下”，以及“族”字及其之间的内容
            area_name = re.sub(rf'^{province}(上|下)?[^族]*族', province, area_name)
            break  # 找到匹配的省份后可以退出循环
    return area_name.strip()  # 返回清洗后的名称



# 匹配函数，增加对 name 的记录
def match_area(area, name):
    match = process.extractOne(area, reference_df['combined'])
    return name, area, match[0], match[1]  # 返回 name，原始地名，匹配结果和相似度

# 读取地名文件
area_df = pd.read_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales/地图匹配_0426_v4.csv')  # 包含不规范的地名
reference_df = pd.read_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241105area_value_combinations.csv')  # 包含所有可能的地名

columns_to_check = [
    "PROVGB_1953", "CITYGB_1953", "CNTYGB_1953",
    "PROVGB_1964", "CITYGB_1964", "CNTYGB_1964",
    "PROVGB_1982", "CITYGB_1982", "CNTYGB_1982",
    "PROVGB_1990a", "CITYGB_1990a", "CNTYGB_1990a",
    "PROVGB_2000", "CITYGB_2000", "CNTYGB_2000",
    "PROVGB_2010", "CITYGB_2010", "CNTYGB_2010"
]

# 保留所有这些列都为空的行
area_df = area_df[area_df[columns_to_check].isnull().all(axis=1)]


# 新增 nozu_area 列
area_df['nozu_area'] = area_df['area'].apply(lambda x: remove_province_and_zu(x, provinces))

# 清洗地名
tqdm.pandas(desc="清洗地名")
area_df['cleaned_area'] = Parallel(n_jobs=-1)(
    delayed(clean_area_name)(area) for area in tqdm(area_df['nozu_area'], desc="清洗地名")
)

reference_df['combined'] = Parallel(n_jobs=-1)(
    delayed(clean_area_name)(f"{row['CPROV']}{row['CCNTY']}") for _, row in reference_df.iterrows()
)

# 匹配地名并记录结果
tqdm.pandas(desc="匹配地名")
results = Parallel(n_jobs=-1)(
    delayed(match_area)(row['cleaned_area'], row['name']) for _, row in tqdm(area_df.iterrows(), total=area_df.shape[0], desc="匹配地名")
)

# 将结果转换为 DataFrame
results_df = pd.DataFrame(results, columns=['name', 'cleaned_original_area', 'matched_area', 'similarity'])

# 保存结果到 CSV 文件
results_df.to_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/matched_areas.csv', index=False)

匹配地名: 100%|██████████| 3348/3348 [00:50<00:00, 66.42it/s]


In [31]:
import pandas as pd

# 读取 CSV 文件
matched_areas_df = pd.read_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/matched_areas.csv')
raw_data_tales_df = pd.read_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/data/raw_data_tales/地图匹配_0426_v4.csv')

# 合并数据，依据 "name" 列
merged_df = pd.merge(matched_areas_df, raw_data_tales_df[['name', 'area']], on='name', how='left')

# 保存结果到新的 CSV 文件
merged_df.to_csv('/Users/zhaorunping/Desktop/Research_Onging/2410_LSE_Xue/result/241105cleaned_matched_areas.csv', index=False)
